In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
sabribelmadoui_arabic_sign_language_unaugmented_dataset_path = kagglehub.dataset_download('sabribelmadoui/arabic-sign-language-unaugmented-dataset')

print('Data source import complete.')


Using Colab cache for faster access to the 'arabic-sign-language-unaugmented-dataset' dataset.
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Streaming output truncated to the last 5000 lines.
/kaggle/input/arabic-sign-language-unaugmented-dataset/unaugmented/416/train/labels/IMG_20210615_163933_jpg.rf.5c5699969615feeeaa6c4fbaf8ea23a0.txt
/kaggle/input/arabic-sign-language-unaugmented-dataset/unaugmented/416/train/labels/IMG_20210623_165047_jpg.rf.9a261f34023f9274a9ad766c94d33cd7.txt
/kaggle/input/arabic-sign-language-unaugmented-dataset/unaugmented/416/train/labels/IMG_20210610_180718_jpg.rf.507bf65fd7fdbf27a27fd1af1a247865.txt
/kaggle/input/arabic-sign-language-unaugmented-dataset/unaugmented/416/train/labels/IMG_20210612_172929_jpg.rf.15b0fa43f1c4ce4474964a01123382cf.txt
/kaggle/input/arabic-sign-language-unaugmented-dataset/unaugmented/416/train/labels/IMG_20210611_211744_jpg.rf.d7cb5846aa4673f89ec8f2d205927b84.txt
/kaggle/input/arabic-sign-language-unaugmented-dataset/unaugmented/416/train/labels/IMG_20210624_230741_jpg.rf.e070d5d5eaca9e2848ddcebef2061b61.txt
/kaggle/input/arabic-sign-language-unaugmented-dataset/unaugm

In [ ]:
pip install tensorflow mediapipe

# Import libraries

In [ ]:
import os
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)



# Image preprocessing

In [ ]:

DATA_DIR = '/kaggle/input/arabic-sign-language-unaugmented-dataset/unaugmented/416/'

data = []
labels = []
img_pathes = ["train", "test", "valid"]
for image_path in img_pathes:
  dict_image_path = os.path.join(DATA_DIR, image_path, 'images')
  dict_label_path = os.path.join(DATA_DIR, image_path, 'labels')
  for entry_name in os.listdir(dict_image_path):
      img_path = os.path.join(dict_image_path, entry_name)
      label_path = os.path.join(dict_label_path, entry_name[:-3] + 'txt')
      if not os.path.exists(label_path):
        entry_name = re.sub(r'\(\d+\)', '', entry_name)
        label_path = os.path.join(dict_label_path, entry_name[:-3] + 'txt')

      with open(label_path, 'r') as f:
        alphabet_label = f.readline().split()[0]

      data_aux = []
      img = cv2.imread(os.path.join(img_path))
      img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

      results = hands.process(img_rgb)
      if results.multi_hand_landmarks:
          for hand_landmarks in results.multi_hand_landmarks:
            for landmark_ in (hand_landmarks.landmark):
                x = landmark_.x
                y = landmark_.y
                data_aux.append(x)
                data_aux.append(y)
            data.append(data_aux)
            labels.append(int(alphabet_label))
print(len(data))
print(len(labels))

5734
5734


# Saving data

In [ ]:
f = open('data.pkl', 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()

# Split the data

In [ ]:
data_dict = pickle.load(open('./data.pkl', 'rb'))
set_numbers = set()
for i in (data_dict["data"]):
  set_numbers.add(len(i))
print(set_numbers)

{42, 84}


In [ ]:
import numpy as np
import pickle

data_dict = pickle.load(open('./data.pkl', 'rb'))
data = data_dict['data']
labels = data_dict['labels']

max_len = max(len(seq) for seq in data)

padded_data = []
for seq in data:
    padded_seq = seq + [0] * (max_len - len(seq))
    padded_data.append(padded_seq)

data = np.asarray(padded_data)
labels = np.asarray(labels)

print("Shape of data after padding:", data.shape)
print("Shape of labels:", labels.shape)

Shape of data after padding: (5734, 84)
Shape of labels: (5734,)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42, shuffle=True, stratify=labels)


# Modeling

In [ ]:
model = RandomForestClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
score = accuracy_score(y_predict, y_test)
print('{}% of samples were classified correctly !'.format(score * 100))

83.60941586748038% of samples were classified correctly !


In [ ]:
model = XGBClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
score = accuracy_score(y_predict, y_test)
print('{}% of samples were classified correctly !'.format(score * 100))

88.92763731473408% of samples were classified correctly !


In [ ]:
f = open('model.p', 'wb')
pickle.dump({'model': model}, f)
f.close()

# Testing

In [ ]:
cam = cv2.VideoCapture(0)
# names: ['ALIF', 'BAA', 'TA', 'THA', 'JEEM', 'HAA', 'KHAA', 'DELL', 'DHELL', 'RAA', 'ZAY', 'SEEN', 'SHEEN', 'SAD', 'DAD', 'TAA', 'DHAA',
#  'AYN', 'GHAYN', 'FAA', 'QAAF', 'KAAF', 'LAAM', 'MEEM', 'NOON', 'HA', 'WAW', 'YA']

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

letters = {
    0: "أ", 1: "ب", 2: "ت", 3: "ث", 4: "ج",
    5: "ح", 6: "خ", 7: "د", 8: "ذ", 9: "ر",
    10: "ز", 11: "س", 12: "ش", 13: "ص", 14: "ض",
    15: "ط", 16: "ظ", 17: "ع", 18: "غ", 19: "ف",
    20: "ق", 21: "ك", 22: "ل", 23: "م", 24: "ن",
    25: "ه", 26: "و", 27: "ي"
}

while True:

    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cam.read()

    # Check if the frame was read successfully
    if not ret or frame is None:
        print("Failed to capture frame from webcam. Exiting...")
        break


    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10

        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10

        prediction = model.predict([np.asarray(data_aux)])

        predicted_character = letters[int(prediction[0])]

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                    cv2.LINE_AA)

    cv2.imshow('frame', frame)
    cv2.waitKey(1)


cam.release()
cv2.destroyAllWindows()

Failed to capture frame from webcam. Exiting...
